In [1]:
import ROOT as rt
import root_numpy as rtnp
import numpy as np
from matplotlib import pyplot as plt
from histo_utilities import create_TH1D, create_TH2D, std_color_list

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 1
CMS_lumi.extraText = "     Simulation Preliminary"

donotdelete = []
import csv
import re
import sys

Welcome to JupyROOT 6.13/02


In [3]:
fpath = {}
tree = {}
# fpath['mh125'] = '/home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_mh125_mx50_pl1000_ev10000.root'
fpath['mh1000'] = '/home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withISR/jet_timing_studies_ntuple_ISR_mh1000_mx475_pl1000_ev10000.root'

# fpath['vbfh'] = '/home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withoutISR/jetNtuple_glueball/jet_timing_studies_ntuple_vbfh_mh125_mx50_pl1000_ev10000.root'

for k,v in fpath.iteritems():
    print k, v
    tree[k] = rtnp.root2array(v)
    #tree[k] = rtnp.tree2array(v)
    #jet_max_pt_array = [np.max(x) for x in tree[k]["jetPt"]]
    a = tree[k]["jetPt"]
    print a[:-1][0]
    
    
trigger_names_file = '/home/cms/Desktop/LLP/delayed_jet_analyzer/data/trigger_names_llp_v1.dat'
trigger_names = []
with open(trigger_names_file) as f:
    reader = csv.reader(f, delimiter=" ")
    for line in reader:
        trigger_names.append(line[2])

mh1000 /home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withISR/jet_timing_studies_ntuple_ISR_mh1000_mx475_pl1000_ev10000.root


IOError: cannot read /home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withISR/jet_timing_studies_ntuple_ISR_mh1000_mx475_pl1000_ev10000.root

Error in <TFile::TFile>: file /home/cms/Desktop/LLP/jetNtuple_RunIISummer16/withISR/jet_timing_studies_ntuple_ISR_mh1000_mx475_pl1000_ev10000.root does not exist


In [ ]:
trigger_paths = [84,136,196,310] #PFMET120

In [ ]:
# inclusive trigger efficiency * acceptance
trigger_decision = {}
mu_bool = []
ele_bool = []

higgs_pt = []
llp_pt = []

T=tree['vbfh']
gParticleId = T['gParticleId']
gParticleMotherIndex = T['gParticleMotherIndex']
gParticleStatus = T['gParticleStatus']
gLLP_daughter_pt = T['gLLP_daughter_pt']
gLLP_daughter_eta = T['gLLP_daughter_eta']
gLLP_daughter_phi = T['gLLP_daughter_phi']
gLLP_daughter_e = T['gLLP_daughter_e']
hlt = T['HLTDecision']
# weight = T['weight']
nEvents = len(T['eventNum'])
for tr in trigger_paths:
    trigger_decision[tr] = []
for i in range(nEvents):
    higgs_vec = rt.TLorentzVector()
    llp1_vec = rt.TLorentzVector()
    llp2_vec = rt.TLorentzVector()
    for j in range(4):
        temp = rt.TLorentzVector()
        temp.SetPtEtaPhiE(gLLP_daughter_pt[i][j],gLLP_daughter_eta[i][j],gLLP_daughter_phi[i][j],gLLP_daughter_e[i][j])
        higgs_vec += temp
        if j<=1:
            llp1_vec += temp
        else:
            llp2_vec += temp
    llp_pt.append(max(llp1_vec.Pt(),llp2_vec.Pt()))
    higgs_pt.append(higgs_vec.Pt())
    for tr in trigger_paths:
        trigger_decision[tr].append(hlt[i][tr])
    #choose only events with 1 muon
    mu = np.logical_and(np.logical_and(np.absolute(gParticleId[i]) == 13,  gParticleMotherIndex[i] == 1), gParticleStatus[i] == 1 )
#     if np.count_nonzero(mu) <= 2 and np.count_nonzero(mu) > 0: #has 1 or 2 muon
    if np.count_nonzero(mu) <=2 and np.count_nonzero(mu)>0:
        mu_bool.append(True)
    else:
        mu_bool.append(False)
    ele = np.logical_and(np.logical_and(np.absolute(gParticleId[i]) == 11,  gParticleMotherIndex[i] == 1), gParticleStatus[i] == 1 )
    if np.count_nonzero(ele) <= 2 and np.count_nonzero(ele) > 0: #has 1 or 2 ele
        ele_bool.append(True)
    else:
        ele_bool.append(False)
# weights = weight*140 #corresponding to 140 pb
higgs_pt = np.array(higgs_pt)
mu_bool = np.array(mu_bool)
llp_pt = np.array(llp_pt)
for tr in trigger_paths:
    trigger_decision[tr] = np.array(trigger_decision[tr])
assert(len(mu_bool) == len(llp_pt))

In [ ]:
print(np.count_nonzero(trigger_decision[310]),trigger_decision[310].shape)

In [ ]:
# Plotting Reco Time
c = rt.TCanvas('c','c', 800, 600)
h = {}
leg = rt.TLegend(0.68,0.70,0.97,0.93);
leg.SetTextSize(0.03)
condition = ele_bool
h['mh125'] = create_TH1D( llp_pt, axis_title=['LLP pT [GeV]', 'events'], name=k, binning=[40,0,800])
#     leg.AddEntry(h[k],legend[k])
h['mh125'].Draw()
# leg.Draw()
c.SetLogy()
c.Draw()
c.SaveAs("../plots/trigger_efficiency/vbfh_mh125_mx50_pl1000_llppt.pdf")
# c.SaveAs("../plots/trigger_efficiency/overall_mh125_mx50_pl1000_llppt.pdf")

In [ ]:
# Plotting Reco Time
c = rt.TCanvas('c','c', 800, 600)
h = {}
leg = rt.TLegend(0.68,0.70,0.97,0.93);
leg.SetTextSize(0.03)

h['mh125'] = create_TH1D( higgs_pt, axis_title=['Higgs pT [GeV]', 'events'], name=k, binning=[40,0,800],weights=weight)
#     leg.AddEntry(h[k],legend[k])
h['mh125'].Draw()
# leg.Draw()
c.SetLogy()
c.Draw()
c.SaveAs("../../plots/trigger_efficiency/higgs_pt_allmode.pdf")
# c.SaveAs("../plots/trigger_efficiency/overall_mh125_mx50_pl1000_PFMET120.pdf")

In [ ]:
# Plotting Reco Time
c = rt.TCanvas('c','c', 800, 600)
h = {}
leg = rt.TLegend(0.68,0.70,0.97,0.93);
leg.SetTextSize(0.03)
# condition = np.logical_or(ele_bool,mu_bool)
condition = np.ones(llp_pt.shape, dtype=int)

trig = trigger_decision[trigger_paths[3]]
# trig = np.logical_or(trigger_decision[trigger_paths[0]],trigger_decision[trigger_paths[2]])[condition]
xaxis = llp_pt
xaxis_title = 'LLP pT[GeV]'
# bins = [0,10,20,30,40,50,60,70,80,90,100,150,200,250]
# bins = [0,20,40,60,80,100,125,150,175,200,250,300]
bins = [0,10,20,30,40,50,60,70,80,90,100,125,150,175,200,250,300]
bins = [50,0,500]
hm = create_TH1D(xaxis, 'hm2', axis_title = [xaxis_title,'Trigger Efficiency'], binning=bins,weights = weight*trig)
hb = create_TH1D(xaxis, 'hb2', axis_title = [xaxis_title,'Trigger Efficiency'], binning=bins,weights = weight)
pEff = rt.TEfficiency(hm,hb)
for pt in xaxis:
    if pt>400:
        print pt

# trig = trigger_decision[trigger_paths[3]]
# xaxis = llp_pt
# bins = [0,20,40,60,80,100,125,150,175,200,250,300]

# hm = create_TH1D(xaxis, 'hm2', axis_title = [xaxis_title,'Trigger Efficiency'], binning=bins,weights =trig)
# hb = create_TH1D(xaxis, 'hb2', axis_title = [xaxis_title,'Trigger Efficiency'], binning=bins)
# pEff = rt.TEfficiency(hm,hb)
#     leg.AddEntry(h[k],legend[k])
pEff.Draw()
# leg.Draw()
c.Draw()
# c.SaveAs("../plots/trigger_efficiency/vbfh_mh125_mx50_pl1000_trig"+str(trigger_paths[3])+".pdf")

# print(weight[mu_bool]*trigger_decision[tr][mu_bool])